# Download Data using OPeNDAP
### By Minh Phan

Open-source Project for a Network Data Access Protocol (OPeNDAP) is the developer of client/server software of the same name, enabling scientists to share data easily over the internet ([EarthData](https://www.earthdata.nasa.gov/engage/open-data-services-and-software/api/opendap)). Using an OPeNDAP URL of any database server, we can access data easily and stream it directly and seamlessly to your local machine. This tutorial is based on [this notebook from the Copernicus Marine Data](https://help.marine.copernicus.eu/en/articles/4854800-how-to-open-and-visualize-copernicus-marine-data-using-python), with some modifications to reflect our workflow and demands outlined in the previous notebooks.

## Import necessary libraries

In [1]:
import xarray as xr
import getpass

At the moment, implementing using .netrc is not probable for some reason. Will retry later.

In [6]:
USERNAME = 'mphan'
PASSWORD = getpass.getpass('Enter your password: ')

Enter your password:  ········


In [7]:
# change your Dataset ID accordingly
DATASET_ID = 'cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m'

In [8]:
def copernicusmarine_datastore(dataset, username, password):
    from pydap.client import open_url
    from pydap.cas.get_cookies import setup_session
    cas_url = 'https://cmems-cas.cls.fr/cas/login'
    session = setup_session(cas_url, username, password)
    session.cookies.set("CASTGC", session.cookies.get_dict()['CASTGC'])
    database = ['my', 'nrt']
    url = f'https://{database[0]}.cmems-du.eu/thredds/dodsC/{dataset}'
    try:
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits 
    except:
        url = f'https://{database[1]}.cmems-du.eu/thredds/dodsC/{dataset}'
        data_store = xr.backends.PydapDataStore(open_url(url, session=session, user_charset='utf-8')) # needs PyDAP >= v3.3.0 see https://github.com/pydap/pydap/pull/223/commits
    return data_store

In [9]:
data_store = copernicusmarine_datastore(DATASET_ID, USERNAME, PASSWORD)

When you open a dataset from PyDAP's Data Store object, only the "shell" of the dataset is streamed into our local machine. The rest of the data shall be remote, so if we want to process further outside of slicing/inspecting, we need to stream the additional data, which occupies lots of memory. The streaming/downloading speed is also very slow, so proceed with caution. 

In [10]:
DS = xr.open_dataset(data_store)

In [11]:
DS

<xarray.Dataset>
Dimensions:    (longitude: 4320, latitude: 2041, depth: 50, time: 999)
Coordinates:
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
  * latitude   (latitude) float32 -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * depth      (depth) float32 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * time       (time) datetime64[ns] 2020-11-01T12:00:00 ... 2023-07-27T12:00:00
Data variables:
    thetao     (time, depth, latitude, longitude) float32 ...
Attributes:
    producer:     CMEMS - Global Monitoring and Forecasting Centre
    title:        daily mean fields from Global Ocean Physics Analysis and Fo...
    area:         Global
    Conventions:  CF-1.8
    credit:       E.U. Copernicus Marine Service Information (CMEMS)
    contact:      https://marine.copernicus.eu/contact
    references:   http://marine.copernicus.eu
    source:       MOI GLO12
    licence:      http://marine.copernicus.eu/services-portfolio/service-comm...
    institution:  Mercator Ocean International